Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.417422
Training accuracy: 9.4%
Validation accuracy: 11.6%
Loss at step 100: 2.369087
Training accuracy: 71.6%
Validation accuracy: 69.6%
Loss at step 200: 1.903106
Training accuracy: 74.6%
Validation accuracy: 72.2%
Loss at step 300: 1.646330
Training accuracy: 75.9%
Validation accuracy: 73.2%
Loss at step 400: 1.473612
Training accuracy: 76.7%
Validation accuracy: 73.8%
Loss at step 500: 1.347282
Training accuracy: 77.3%
Validation accuracy: 74.0%
Loss at step 600: 1.249783
Training accuracy: 77.8%
Validation accuracy: 74.3%
Loss at step 700: 1.171439
Training accuracy: 78.5%
Validation accuracy: 74.7%
Loss at step 800: 1.106466
Training accuracy: 79.0%
Validation accuracy: 74.8%
Test accuracy: 82.7%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.367050
Minibatch accuracy: 12.5%
Validation accuracy: 14.6%
Minibatch loss at step 500: 1.695278
Minibatch accuracy: 74.2%
Validation accuracy: 75.1%
Minibatch loss at step 1000: 1.463533
Minibatch accuracy: 78.9%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 0.879357
Minibatch accuracy: 81.2%
Validation accuracy: 76.3%
Minibatch loss at step 2000: 1.223464
Minibatch accuracy: 77.3%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 1.068735
Minibatch accuracy: 82.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 1.119114
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Test accuracy: 85.7%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [8]:
print(train_dataset.shape)
print(28**2)

(200000, 784)
784


In [9]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
    
  # first the hidden layer
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

  # now the output layer
  biases_out = tf.Variable(tf.zeros([num_labels]))
  weights_out = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))  
    
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(hidden, weights_out) + biases_out
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
    
  valid_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

  test_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

In [10]:
num_steps = 200

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 20 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 315.200836
Training accuracy: 13.3%
Validation accuracy: 28.6%
Loss at step 20: 36.145287
Training accuracy: 81.8%
Validation accuracy: 79.2%
Loss at step 40: 18.220436
Training accuracy: 84.0%
Validation accuracy: 78.9%
Loss at step 60: 10.086283
Training accuracy: 86.9%
Validation accuracy: 79.2%
Loss at step 80: 6.504560
Training accuracy: 89.2%
Validation accuracy: 79.3%
Loss at step 100: 4.857653
Training accuracy: 91.8%
Validation accuracy: 79.6%
Loss at step 120: 2.601049
Training accuracy: 93.3%
Validation accuracy: 79.2%
Loss at step 140: 1.014773
Training accuracy: 96.8%
Validation accuracy: 79.4%
Loss at step 160: 0.430486
Training accuracy: 98.2%
Validation accuracy: 79.2%
Loss at step 180: 23.938372
Training accuracy: 83.3%
Validation accuracy: 79.2%
Test accuracy: 86.4%


# Now try do the above with batch

In [12]:
batch_size = 400

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
    
  # first the hidden layer
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

  # now the output layer
  biases_out = tf.Variable(tf.zeros([num_labels]))
  weights_out = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))  
    
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(hidden, weights_out) + biases_out
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  #l2_reg = tf.nn.l2_
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
    
  valid_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

  test_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

In [13]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 331.288330
Minibatch accuracy: 6.0%
Validation accuracy: 29.0%
Minibatch loss at step 500: 9.123020
Minibatch accuracy: 82.5%
Validation accuracy: 82.5%
Minibatch loss at step 1000: 7.950509
Minibatch accuracy: 84.2%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 6.028342
Minibatch accuracy: 82.0%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 3.481837
Minibatch accuracy: 84.2%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 4.018651
Minibatch accuracy: 86.0%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 3.913892
Minibatch accuracy: 83.0%
Validation accuracy: 82.4%
Minibatch loss at step 3500: 1.724365
Minibatch accuracy: 86.5%
Validation accuracy: 84.5%
Minibatch loss at step 4000: 1.431915
Minibatch accuracy: 85.5%
Validation accuracy: 85.1%
Minibatch loss at step 4500: 1.752607
Minibatch accuracy: 87.5%
Validation accuracy: 85.1%
Test accuracy: 90.9%


# Now see if we can replicate this with keras

In [15]:
import keras as k

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

In [42]:
model = Sequential()
model.add(Dense(1024, input_dim=image_size ** 2, bias=True, activation='relu', init='he_normal'))
model.add(Dense(10, bias=True, activation='softmax', init='he_normal'))

optimizer = SGD(lr=0.5, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [43]:
model.fit(train_dataset, train_labels, nb_epoch=10, batch_size=400)

Epoch 1/10
200000/200000 [==============================] - 22s - loss: 0.5390 - acc: 0.8459    
Epoch 2/10
200000/200000 [==============================] - 21s - loss: 0.4013 - acc: 0.8828    
Epoch 3/10
200000/200000 [==============================] - 22s - loss: 0.3530 - acc: 0.8965    
Epoch 4/10
200000/200000 [==============================] - 22s - loss: 0.3212 - acc: 0.9060    
Epoch 5/10
200000/200000 [==============================] - 22s - loss: 0.2948 - acc: 0.9130    
Epoch 6/10
200000/200000 [==============================] - 22s - loss: 0.2735 - acc: 0.9188    
Epoch 7/10
200000/200000 [==============================] - 22s - loss: 0.2548 - acc: 0.9248    
Epoch 8/10
200000/200000 [==============================] - 22s - loss: 0.2358 - acc: 0.9303    
Epoch 9/10
200000/200000 [==============================] - 22s - loss: 0.2220 - acc: 0.9340    
Epoch 10/10
200000/200000 [==============================] - 22s - loss: 0.2079 - acc: 0.9386    


In [47]:
score_train = model.evaluate(train_dataset, train_labels, batch_size=5000)
print(score_train)
score_test = model.evaluate(test_dataset, test_labels, batch_size=5000)
print(score_test)

200000/200000 [==============================] - 10s    
[0.17525205761194229, 0.94831499457359314]
10000/10000 [==============================] - 0s     
[0.16517212241888046, 0.95320001244544983]


# I like keras!